This document parses and transforms CSV High Frontier 4 patent cards into JSON.

# Preamble

## CSV import

In [1]:
import csv
import glob
from os.path import splitext, basename
import unittest

def read_cards(deckname, fields):
    with open(f'decks/{deckname}.csv', 'r') as fh:
        reader = csv.DictReader(fh, fieldnames=fields)
        next(reader) # Drop row 1, which contains multi-collumn cells describing groups of fields (e.g. Thruster and Support Requirements)
        next(reader) # Drop row 2, which contains field names
        deck = list(reader)
        
    for i, card in enumerate(deck):
        split_keys(card)
        card['Deck'] = deckname
        card['Side'] = side_from_index(i)
                
    return deck    

def side_from_index(i):
    if i % 2 == 0:
        return 'Front'
    return 'Reverse'

def split_keys(card):
    namespaced_keys = [k for k in card if '.' in k]
    for key in namespaced_keys:
        k0, k1 = key.split('.')
        if not card.get(k0):
            card[k0] = {}
        card[k0][k1] = card.pop(key)

def chunklist(l, n):
    return zip(*[iter(l)] * 2)
               
def build_deck(deckname, clean_cards):
    return {
        'Name': deckname,
        'Patents': [
            {
                'Deck': deckname,
                'Front': front,
                'Reverse': reverse,
            }
            for front, reverse in chunklist(clean_cards, 2)
        ]
    }

class TestUtils(unittest.TestCase):
    def test_split(self):
        card = {
            'Mass': 0,
            'Rad Hard': 4,
            'Thrust Triangle.Thrust': 3,
            'Thrust Triangle.Fuel Consumption': '2',
            'Support Requirements.⟛ Generator': '1'
        }
        
        expected_result = {
            'Mass': 0,
            'Rad Hard': 4,
            'Thrust Triangle': {
                'Thrust': 3,
                'Fuel Consumption': '2',
            },
            'Support Requirements': {'⟛ Generator': '1'},
        }
        
        split_keys(card)
        self.assertEqual(card, expected_result)
        
    def test_build_deck(self):
        cards = [1, 2, 3, 4, 5, 6]
        
        deck = build_deck('Test', cards)
        expected_deck = {
            'Name': 'Test',
            'Patents': [
                { 'Deck': 'Test', 'Front': 1, 'Reverse': 2 },
                { 'Deck': 'Test', 'Front': 3, 'Reverse': 4 },
                { 'Deck': 'Test', 'Front': 5, 'Reverse': 6 },
            ]
        }
    
        self.assertEqual(deck, expected_deck)
        
unittest.main(argv=['', 'TestUtils'], verbosity=2, exit=False)

test_build_deck (__main__.TestUtils) ... ok
test_split (__main__.TestUtils) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


## Support requirement parsing

In [2]:
import unittest

def supports(card):
    supports = []
    
    # Special case for Colliding FRC 3He-D Fusion TeraWatt Thruster
    if card.get('Support Requirements', {}).get('Any Reactor', False):
        any_reactor = {'Type': 'Reactor', 'Subtypes': ['∿', '💣', 'X']}
        supports += [any_reactor, any_reactor]      

    reactor_subtypes = []    
    # Other than the Colliding FRC 3He-D Fusion TeraWatt Thruster, cards only need 1 reactor symbol. If we have multiple symbols, that means that we need any one of them, not several.
    for t in ['∿', '💣', 'X']:
        if card.get('Support Requirements', {}).get(f'{t} Reactor', False):
            reactor_subtypes.append(t)

    if len(reactor_subtypes) != 0:
        supports.append(
            {
                'Type': 'Reactor',
                'Subtypes': reactor_subtypes
            }
        )

    # Cards only ever need exactly one generator type
    if card.get('Support Requirements', {}).get('e Generator', False):
        supports.append(
            {
                'Type': 'Generator',
                'Subtypes': ['e']
            }
        )
    elif card.get('Support Requirements', {}).get('⟛ Generator', False):
        supports.append(
            {
                'Type': 'Generator',
                'Subtypes': ['⟛']
            }
        )

    therms = card.get('Support Requirements', {}).get('Cooling', 0)
    if therms != 0:
        supports.append({'Type': 'Cooling', 'Therms': therms})
    
    return supports

class SupportTests(unittest.TestCase):
    
    def test_no_supports(self):
        card = {}
        self.assertEqual(supports(card), [])
        
    def test_single_generator(self):
        card = {'Support Requirements': {'⟛ Generator': True}}
        self.assertEqual(supports(card), [{'Type': 'Generator', 'Subtypes': ['⟛']}])

    def test_single_reactor(self):
        card = {'Support Requirements': {'💣 Reactor': True}}
        self.assertEqual(supports(card), [{'Type': 'Reactor', 'Subtypes': ['💣']}])
        
    def test_multiple_reactors(self):
        card = {'Support Requirements': {'💣 Reactor': True, '∿ Reactor': True, 'X Reactor': True}}
        self.assertEqual(supports(card), [{
            'Type': 'Reactor',
            'Subtypes': ['∿', '💣', 'X']
        }])

    def test_reactor_and_generator(self):
        card = {'Support Requirements': {'💣 Reactor': True, 'X Reactor': False, 'e Generator': True}}
        self.assertEqual(supports(card), [{'Type': 'Reactor', 'Subtypes': ['💣']}, {'Type': 'Generator', 'Subtypes': ['e']}])
        
    def test_therms(self):
        card = {'Support Requirements': {'Cooling': 2}}
        self.assertEqual(supports(card), [{'Type': 'Cooling', 'Therms': 2}])
    
    def test_colliding_frc_special_case(self):
        '''
        The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type support requirements.
        '''
        card = {'Support Requirements': {'Any Reactor': True}}
        any_reactor = {'Type': 'Reactor', 'Subtypes': ['∿', '💣', 'X']}
        self.assertEqual(supports(card), [any_reactor, any_reactor])
        

unittest.main(argv=['', 'SupportTests'], verbosity=2, exit=False)

test_colliding_frc_special_case (__main__.SupportTests)
The Colliding FRC 3He-D Fusion GW Thruster has the 'Any' reactor subtype in the spreadsheet. It actually requires TWO of any reactor. We handle this as a special case which requires two Reactor type support requirements. ... ok
test_multiple_reactors (__main__.SupportTests) ... ok
test_no_supports (__main__.SupportTests) ... ok
test_reactor_and_generator (__main__.SupportTests) ... ok
test_single_generator (__main__.SupportTests) ... ok
test_single_reactor (__main__.SupportTests) ... ok
test_therms (__main__.SupportTests) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.005s

OK


## Debug tools

In [3]:
import random

def draw(deck):
    return random.choice(deck)

# Card imports

## Thrusters

In [4]:
'''
Thrusters
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Fuel Type',
    'Bonus Pivots',
    'Thrust Triangle.Afterburn',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Afterburn',
    'Support Requirements.Cooling',
    'Bonus Pivots',
]

bool_fields = [
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

def str_to_int(s):
    if s == '':
        return 0
    else:
        return int(s)

def str_to_bool(s):
    if s == '0' or s == '':
        return False
    else:
        return True

def intify(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = str_to_int(card[k0][k1])
        else:
            card[k] = str_to_int(card[k])

def boolify(card, fields):
    for k in fields:
        if '.' in k:
            k0, k1 = k.split('.')
            card[k0][k1] = str_to_bool(card[k0][k1])
        else:
            card[k] = str_to_bool(card[k])
            
thrusters = read_cards('Thrusters', fields)
for i, card in enumerate(thrusters):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    
thruster_deck = build_deck('Thrusters', thrusters)

## Robonauts

In [5]:
import unittest

def isru(robonaut):
    return {
        'Rating': robonaut['ISRU']['ISRU'],
        'Platforms': [p for p in ['Raygun', 'Buggy', 'Missile'] if robonaut.get('ISRU').get(p)],
    }

class ISRUTests(unittest.TestCase):
    
    def test_raygun(self):
        card = {'ISRU': {'ISRU': 2, 'Raygun': True, 'Buggy': False, 'Missile': False}}
        self.assertEqual(isru(card), {'Rating': 2, 'Platforms': ['Raygun']})
    
    def test_buggy(self):
        card = {'ISRU': {'ISRU': 0, 'Raygun': False, 'Buggy': True, 'Missile': False}}
        self.assertEqual(isru(card), {'Rating': 0, 'Platforms': ['Buggy']})
        
    def test_missile(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': False, 'Buggy': False, 'Missile': True}}
        self.assertEqual(isru(card), {'Rating': 1, 'Platforms': ['Missile']})

    def test_multiple(self):
        card = {'ISRU': {'ISRU': 1, 'Raygun': True, 'Buggy': False, 'Missile': True}}
        self.assertEqual(isru(card), {'Rating': 1, 'Platforms': ['Raygun', 'Missile']})

unittest.main(argv=['', 'ISRUTests'], verbosity=2, exit=False)

test_buggy (__main__.ISRUTests) ... ok
test_missile (__main__.ISRUTests) ... ok
test_multiple (__main__.ISRUTests) ... ok
test_raygun (__main__.ISRUTests) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.002s

OK


In [6]:
'''
Robonauts
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Fuel Type',
    'Thrust Triangle.Afterburn',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Afterburn',
    'Support Requirements.Cooling',
    'ISRU.ISRU',
]

bool_fields = [
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'ISRU.Missile',
    'ISRU.Buggy',
    'ISRU.Raygun',
 ]

def has_thruster(card):
    return card.get('Thrust Triangle', {}).get('Thrust', '') != ''

robonauts = read_cards('Robonauts', fields)
for i, card in enumerate(robonauts):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    if has_thruster(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Thrust Triangle')
        intify(card, filter(lambda f: not f.startswith('Thrust Triangle.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Thrust Triangle.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['ISRU'] = isru(card)
    
robonaut_deck = build_deck('Thrusters', robonauts)
draw(robonauts)

{'Name': 'Cat Fusion Z-pinch Torch',
 'Spectral Type': 'D',
 'Mass': 0,
 'Rad-Hard': 2,
 'Ability': '',
 'ISRU': {'Rating': 2, 'Platforms': ['Buggy']},
 'Support Requirements': [{'Type': 'Reactor', 'Subtypes': ['X']},
  {'Type': 'Cooling', 'Therms': 1}],
 'Deck': 'Robonauts',
 'Side': 'Front'}

## Refineries

In [7]:
'''
Refineries
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Air Eater',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
]

bool_fields = [
    'Air Eater',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

refineries = read_cards('Refineries', fields)
for i, card in enumerate(refineries):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    
draw(refineries)

{'Name': 'Carbonyl Volatilization',
 'Spectral Type': 'M',
 'Mass': 2,
 'Rad-Hard': 5,
 'Air Eater': False,
 'Ability': 'THORIUM BREEDER: -3 to Colocated size rolls on S Sites.',
 'Support Requirements': [],
 'Deck': 'Refineries',
 'Side': 'Reverse'}

## Reactors

In [8]:
'''
Reactors
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Subtypes.X',
    'Subtypes.∿',
    'Subtypes.💣',
    'Movement Modifiers.Thrust',
    'Movement Modifiers.Fuel Consumption',
    'Air Eater',
    'Support Requirements.⟛ Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Movement Modifiers.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Subtypes.X',
    'Subtypes.∿',
    'Subtypes.💣',
    'Air Eater',
    'Support Requirements.⟛ Generator',
    'Support Requirements.X Reactor',
]

def has_movement_modifiers(card):
    return card.get('Movement Modifiers', {}).get('Thrust', '') != ''

reactors = read_cards('Reactors', fields)
for i, card in enumerate(reactors):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    if has_movement_modifiers(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Movement Modifiers')
        intify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['Subtypes'] = [k for k in card['Subtypes'].keys() if card['Subtypes'][k]]

draw(reactors)

{'Name': 'Ultracold Neutrons',
 'Spectral Type': 'C',
 'Mass': 1,
 'Rad-Hard': 5,
 'Air Eater': False,
 'Ability': 'SCOOP: -2 ISRU for Colocated ISRU platforms at Aerostat Sites.',
 'Subtypes': ['X', '💣'],
 'Support Requirements': [],
 'Deck': 'Reactors',
 'Side': 'Reverse'}

## Radiators

In [9]:
'''
Radiators
'''

fields = [
    'Name',
    'Spectral Type',
    'Light.Mass',
    'Light.Rad-Hard',
    'Light.Therms',
    'Heavy.Mass',
    'Heavy.Rad-Hard',
    'Heavy.Therms',
    'Support Requirements.e Generator',
    'Ability',
]

int_fields = [
    'Light.Mass',
    'Light.Rad-Hard',
    'Light.Therms',
    'Heavy.Mass',
    'Heavy.Rad-Hard',
    'Heavy.Therms',
]

bool_fields = [
    'Support Requirements.e Generator',
]

radiators = read_cards('Radiators', fields)
for i, card in enumerate(radiators):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    intify(card, int_fields)
    boolify(card, bool_fields)
    # The spreadsheet has a single support requirements field but technically the light and heavy side requirements are distinct, so we duplicate it
    support_reqs = supports(card)
    card.pop('Support Requirements')
    card['Light']['Support Requirements'] = support_reqs
    card['Heavy']['Support Requirements'] = support_reqs

draw(radiators)

{'Name': 'Dielectric X-Ray Window',
 'Spectral Type': 'D',
 'Ability': '',
 'Light': {'Mass': 4, 'Rad-Hard': 7, 'Therms': 2, 'Support Requirements': []},
 'Heavy': {'Mass': 5, 'Rad-Hard': 7, 'Therms': 3, 'Support Requirements': []},
 'Deck': 'Radiators',
 'Side': 'Front'}

## Generators

In [ ]:
'''
Generators
'''

fields = [
    'Name',
    'Spectral Type',
    'Mass',
    'Rad-Hard',
    'Subtypes.⟛',
    'Subtypes.e',
    'Movement Modifiers.Thrust',
    'Movement Modifiers.Fuel Consumption',
    'Air Eater',
    'Movement Modifiers.Solar',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Movement Modifiers.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Subtypes.⟛',
    'Subtypes.e',
    'Air Eater',
    'Movement Modifiers.Solar',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.∿ Reactor',
    'Support Requirements.💣 Reactor',
]

generators = read_cards('Generators', fields)
for i, card in enumerate(generators):
    if i % 2 == 0:
        spectral_type = card.get('Spectral Type')
    card['Spectral Type'] = spectral_type
    if has_movement_modifiers(card):
        intify(card, int_fields)
        boolify(card, bool_fields)
    else:
        card.pop('Movement Modifiers')
        intify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), int_fields))
        boolify(card, filter(lambda f: not f.startswith('Movement Modifiers.'), bool_fields))
    card['Support Requirements'] = supports(card)
    card['Subtypes'] = [k for k in card['Subtypes'].keys() if card['Subtypes'][k]]

draw(generators)

## GW Thrusters

In [11]:
'''
GW Thrusters
'''

fields = [
    'Name',
    'Type',
    'Spectral Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Afterburn',
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Any Reactor',
    'Support Requirements.Cooling',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Support Requirements.Cooling',
    'Thrust Triangle.Afterburn',
]

bool_fields = [
    'Support Requirements.⟛ Generator',
    'Support Requirements.e Generator',
    'Support Requirements.X Reactor',
    'Support Requirements.Any Reactor',
]

gwthrusters = read_cards('GW Thrusters', fields)
for i, card in enumerate(gwthrusters):
    if i % 2 == 0:
        card.pop('Future')
        spectral_type = card.get('Spectral Type')
        promotion_colony = card.get('Promotion Colony')
    card['Spectral Type'] = spectral_type
    card['Promotion Colony'] = promotion_colony
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    card['Thrust Triangle']['Fuel Type'] = 'Isotope'
    card['Deck'] = 'GW/TW Thrusters' # Workaround for / in deck name

draw(gwthrusters)

{'Name': 'Amat-Initiated H-B Magnetic-Inertial',
 'Type': 'TW Thruster',
 'Spectral Type': 'S',
 'Promotion Colony': 'D',
 'Mass': 1,
 'Rad-Hard': 10,
 'Future': 'MINI-BLACK HOLE FUTURE: Req = Industrialized centaur with 10 isotope FTs. Effects = double all isotope refuel, 10 VP.',
 'Thrust Triangle': {'Thrust': 8,
  'Fuel Consumption': '0',
  'Afterburn': 5,
  'Fuel Type': 'Isotope'},
 'Support Requirements': [{'Type': 'Generator', 'Subtypes': ['⟛']},
  {'Type': 'Cooling', 'Therms': 1}],
 'Deck': 'GW/TW Thrusters',
 'Side': 'Reverse'}

## Freighters

In [12]:
'''
Freighters
'''

fields = [
    'Name',
    'Type',
    'Spectral Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Load-Limit',
    'Factory Loading Only',
    'Bonus Pivots',
    'Supports Provided.⟛ Generator',
    'Supports Provided.e Generator',
    'Supports Provided.X Reactor',
    'Supports Provided.∿ Reactor',
    'Ability',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Load-Limit',
    'Bonus Pivots',
]

bool_fields = [
    'Factory Loading Only',
    'Supports Provided.⟛ Generator',
    'Supports Provided.e Generator',
    'Supports Provided.X Reactor',
    'Supports Provided.∿ Reactor',
]

freighters = read_cards('Freighters', fields)
for i, card in enumerate(freighters):
    if i % 2 == 0:
        card.pop('Future')
        spectral_type = card.get('Spectral Type')
        promotion_colony = card.get('Promotion Colony')
    card['Spectral Type'] = spectral_type
    card['Promotion Colony'] = promotion_colony
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Supports Provided'] = [k for k in card['Supports Provided'].keys() if card['Supports Provided'][k]]

draw(freighters)

{'Name': 'Fusion Fragment Sail',
 'Type': 'Freighter',
 'Spectral Type': 'V',
 'Promotion Colony': 'M',
 'Mass': 2,
 'Rad-Hard': 1,
 'Load-Limit': 2,
 'Factory Loading Only': False,
 'Bonus Pivots': 1,
 'Ability': 'Immune to flares & radiation belts.',
 'Supports Provided': [],
 'Deck': 'Freighters',
 'Side': 'Front'}

## Bernals

In [13]:
'''
Bernals
'''

fields = [
    'Name',
    'Type',
    'Promotion Colony',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Push',
    'Support Requirements.e Generator',
    'Support Requirements.Cooling',
    'Ability',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Support Requirements.Cooling',
]

bool_fields = [
    'Thrust Triangle.Push',
    'Support Requirements.e Generator',
]

bernals = read_cards('Bernals', fields)
for i, card in enumerate(bernals):
    card.pop('Type')
    if i % 2 == 0:
        promotion_colony = card.get('Promotion Colony')
    card['Promotion Colony'] = promotion_colony
    intify(card, int_fields)
    boolify(card, bool_fields)
    card['Support Requirements'] = supports(card)
    card['Thrust Triangle']['Fuel Type'] = 'Dirt'

draw(bernals)

{'Name': 'L1 Climate Control Bernal',
 'Promotion Colony': 'Atmospheric',
 'Mass': 10,
 'Rad-Hard': 8,
 'Ability': 'HOME: You are always the 1st player, superseding all other claimants.',
 'Thrust Triangle': {'Thrust': 3,
  'Fuel Consumption': '3',
  'Push': True,
  'Fuel Type': 'Dirt'},
 'Support Requirements': [{'Type': 'Generator', 'Subtypes': ['e']}],
 'Deck': 'Bernals',
 'Side': 'Front'}

## Colonists

In [ ]:
from functools import reduce
import unittest

def all_f(*fs):
    return lambda *args, **kwargs: all([f(*args, **kwargs) for f in fs])

class FilterTests(unittest.TestCase):
    
    def test_filtering(self):
        fields = [
            'Name',
            'Type',
            'Thrust Triangle.Thrust',
            'Thrust Triangle.Fuel Consumption',
            'Supports.X Reactor',
            'Supports.∿ Reactor',
            'Supports.💣 Reactor',
            'Ability',
        ]
        
        expected_fields = [
            'Name',
            'Type',
            'Ability',
        ]
        
        filter_functions = [lambda x: not x.startswith('Thrust Triangle.'), lambda x: not x.startswith('Supports.')]
        
        filtered_fields = list(filter(all_f(*filter_functions), fields))
        self.assertEqual(filtered_fields, expected_fields)
    
unittest.main(argv=['', 'FilterTests'], verbosity=2, exit=False)

In [ ]:
'''
Colonists
'''

from functools import reduce

fields = [
    'Name',
    'Type',
    'Specialty',
    'Spectral Type',
    'Promotion Colony',
    'Ideology',
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Fuel Consumption',
    'Thrust Triangle.Fuel Type',
    'Thrust Triangle.Afterburn',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Air Eater',
    'Bonus Pivots',
    'ISRU.ISRU',
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
    'Supports.💣 Reactor',
    'Ability',
    'Future',
]

int_fields = [
    'Mass',
    'Rad-Hard',
    'Thrust Triangle.Thrust',
    'Thrust Triangle.Afterburn',
    'ISRU.ISRU',
    'Bonus Pivots',
]

bool_fields = [
    'ISRU.Missile',
    'ISRU.Raygun',
    'ISRU.Buggy',
    'Thrust Triangle.Push',
    'Thrust Triangle.Solar',
    'Air Eater',
    'Supports.X Reactor',
    'Supports.∿ Reactor',
    'Supports.💣 Reactor',
]

def has_isru(card):
    return card.get('ISRU', {}).get('ISRU', '') != ''

def na_to_empty(card):
    na_fields = [f for f, v in card.items() if v == 'n/a']
    for f in na_fields:
        card[f] = ''

def get_multirow_values(card):
    return {
        'Type': card['Type'],
        'Specialty': card['Specialty'],
        'Promotion Colony': card['Promotion Colony'],
        'Ideology': card['Ideology']
    }

def set_multirow_values(card, values):
    for k, v in values.items():
        card[k] = v
    
colonists = read_cards('Colonists', fields)
for i, card in enumerate(colonists):
    if i % 2 == 0:
        multirow_values = get_multirow_values(card)
    if i % 2 == 1:
        set_multirow_values(card, multirow_values)
        
    filters = []
    if not has_thruster(card):
        card.pop('Thrust Triangle')
        filters.append(lambda x: not x.startswith('Thrust Triangle.'))
    this_has_isru = has_isru(card)
    if not this_has_isru:
        card.pop('ISRU')
        filters.append(lambda x: not x.startswith('ISRU.'))
    
    na_to_empty(card)
    intify(card, filter(all_f(*filters), int_fields))
    boolify(card, filter(all_f(*filters), bool_fields))
    card['Supports'] = [k for k in card['Supports'].keys() if card['Supports'][k]]
    
    if this_has_isru:
        card['ISRU'] = isru(card)
    
    if card['Type'] == 'Robot':
        card.pop('Ideology')
    elif card['Type'] == 'Human':
        card.pop('Spectral Type')

draw(colonists)

# Validation

## Aggregating card lists

In [16]:
from itertools import chain

all_cards = list(chain(thrusters, robonauts, refineries, reactors, radiators, generators, gwthrusters, freighters, bernals, colonists))

## "Leaf" schemas and validation

These schemas are "leaves" in the sense that they don't refer to any other schemas.

In [17]:
from jsonschema import validate, ValidationError
import json
from os import path

def save_schema(schema):
    schema_location = 'https://raw.githubusercontent.com/ouroboros8/high-frontier-cards/main/schema/'
    filename = schema['title'].lower().replace(' ', '-').replace('/', '-') + '.schema.json'
    
    filepath = path.join('schema', filename)
    uri = path.join(schema_location, filename)
    
    schema['$id'] = uri
    with open(filepath, 'w') as f:
        json.dump(schema, f, ensure_ascii=False, indent=2, sort_keys=True)

thrust_triangle_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thrust Triangle',
    'description': 'A thrust triangle on a card.',
    'type': 'object',
    'properties': {
        'Thrust': {
            'description': 'The base thrust of the thrust triangle',
            'type': 'integer',
        },
        'Fuel Consumption': {
            'description': 'A string representation of the base fuel consumption as an integer or fraction.',
            'type': 'string',
        },
        'Fuel Type': {
            'description': 'The type of fuel the thrust triangle consumes.',
            'enum': ['Water', 'Dirt', 'Isotope'],
        },
        'Afterburn': {
            'description': 'The number of afterburns the thruster can perform.',
            'type': 'integer',
        },
        'Push': {
            'description': 'Whether the thrust triangle is pushable.',
            'type': 'boolean',
        },
        'Solar': {
            'description': 'Whether the thrust triangle is solar.',
            'type': 'boolean',
        }
    },
    'required': ['Thrust', 'Fuel Consumption', 'Fuel Type'],
}
save_schema(thrust_triangle_schema)

isru_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'ISRU',
    'description': "The card's ISRU rating and platform icons.",
    'type': 'object',
    'properties': {
        'Rating': {
            'description': 'The ISRU rating.',
            'type': 'integer',
        },
        'Platforms': {
            'description': 'An array of platforms icons on the card.',
            'type': 'array',
            'items': {
                'enum': ['Raygun', 'Buggy', 'Missile'],                
            },
        },
    },
    'required': ['Rating', 'Platforms'],
}
save_schema(isru_schema)

generator_support_requirement_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Generator Support Requirement',
    'description': "A generator support required by a card.",
    'properties': {
        'Type': { 'const': 'Generator' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['e', '⟛'] }
        },
    },
}
save_schema(generator_support_requirement_schema)

reactor_support_requirement_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Reactor Support Requirement',
    'description': "A reactor support required by a card.",    
    'properties': {
        'Type': { 'const': 'Reactor' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['X', '∿', '💣'] }
        },
    },
}
save_schema(reactor_support_requirement_schema)

cooling_support_requirement_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Cooling Support Requirement',
    'description': "A cooling support indicating therms of cooling required by a card.",
    'properties': {
        'Type': { 'const': 'Cooling' },
        'Therms': { 'type': 'integer' },
    },
}

save_schema(cooling_support_requirement_schema)

## Thrusters

In [18]:
from jsonschema import validate, ValidationError
import json

thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thruster',
    'description': "One of the sides of a patent from the Thruster deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Thrusters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Bonus Pivots' : { 'type': 'integer' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements'],
    'additionalProperties': False,
}

save_schema(thruster_schema)

for card in thrusters:
    try:
        print('Validating Thruster:', card['Name'])
        validate(card, thruster_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Thruster: Ablative Plate
Validating Thruster: Ablative Nozzle
Validating Thruster: De Laval Nozzle
Validating Thruster: Magnetic Nozzle
Validating Thruster: Dumbo
Validating Thruster: Timberwind
Validating Thruster: Hall Effect
Validating Thruster: Ion Drive
Validating Thruster: Mass Driver
Validating Thruster: MPD T-wave
Validating Thruster: Metastable Helium
Validating Thruster: n-6Li Microfission
Validating Thruster: Monoatomic Plug Nozzle
Validating Thruster: Vortex Confined Nozzle
Validating Thruster: Photon Heliogyro
Validating Thruster: Electric Sail
Validating Thruster: Photon Kite Sail
Validating Thruster: Mag Sail
Validating Thruster: Ponderomotive VASIMR
Validating Thruster: Pulsed Plasmoid
Validating Thruster: Pulsed Inductive
Validating Thruster: Dual-Stage 4-Grid
Validating Thruster: Re Solar Moth
Validating Thruster: Colliding Beam H-B Fusion
ok


In [19]:
bad_card = {'Name': 'Monoatomic Plug Nozzle',
 'Spectral Type': 'M',
 'Mass': 0,
 'Rad-Hard': 6,
 'Bonus Pivots': 0,
 'Ability': '',
 'Thrust Triangle': {'Thrust': 4,
  'Fuel Consumption': '3',
  'Fuel Type': 'Water',
  'Afterburn': 1,
  'Push': False,
  'Solar': False},
 'Support Requirements': [{'Type': 'Reactor', 'Subtypes': ['OHNO']}],
 'Deck': 'Thrusters',
 'Side': 'Front'}

#validate(bad_card, thruster_schema)

In [20]:
from jsonschema import validate

schema_a = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'A',
    'type': 'object',
    'properties': {
        'value': { 'const': 'A' },
    },
}

validate({ 'value': 'A' }, schema_a)

schema_b = {
    'schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'B',
    'type': 'object',
    'properties': {
        'value': { 'const': 'B' },
    },
}

validate({ 'value': 'B' }, schema_b)

either_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'AorB',
    'type': 'object',
    '$defs': {
        'A': schema_a,
        'B': schema_b,
    },
    'oneOf': [
        { '$ref': '#/$defs/A' },
        { '$ref': '#/$defs/B' },
    ]
}

validate({ 'value': 'A' }, either_schema)

## Patents

In [21]:
from jsonschema import validate

thruster_patent_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thruster Patent',
    'description': 'A card from the Thruster patent deck',
    'properties': {
        'Deck': { 'const': 'Thrusters' },
        'Front': { '$ref': thruster_schema['$id'] },
        'Reverse': { '$ref': thruster_schema['$id'] },
    },
    'required': ['Deck', 'Front', 'Reverse'],
    'additionalProperties': False,
}
save_schema(thruster_patent_schema)

thruster_patent_deck_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Thruster Patent Deck',
    'description': "The thruster patent deck.",
    'properties': {
        'Name': { 'type': 'string' },
        'Patents': {
            'type': 'array',
            'items': { '$ref': thruster_patent_schema['$id'] }
        }
    },
    'required': ['Name', 'Patents'],
    'additionalProperties': False,
}
save_schema(thruster_patent_deck_schema)

patent_deck_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Patent Deck',
    'description': "A patent deck.",
    'oneOf': [
        { '$ref': thruster_patent_deck_schema['$id'] },
    ],
}
save_schema(patent_deck_schema)

validate(thruster_deck, patent_deck_schema)
print('ok')

ok


## Robonaut

In [22]:
from jsonschema import validate, ValidationError
import json

robonaut_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Robonaut',
    'description': "One of the sides of a patent from the Robonaut deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Robonauts' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'ISRU' : { '$ref': isru_schema['$id'] },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'ISRU', 'Support Requirements'],
    'additionalProperties': False,
}

save_schema(robonaut_schema)

for card in robonauts:
    try:
        print('Validating Robonauts:', card['Name'])
        validate(card, robonaut_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Robonauts: Blackbody-Pumped Laser
Validating Robonauts: Fissile Aerosol Laser
Validating Robonauts: Cat Fusion Z-pinch Torch
Validating Robonauts: H-B Cat Inertial
Validating Robonauts: Flywheel Tractor
Validating Robonauts: Electrophoretic Sandworm
Validating Robonauts: Free Electron Laser
Validating Robonauts: Wakefield e-Beam
Validating Robonauts: Kuck Mosquito
Validating Robonauts: Ablative Laser
Validating Robonauts: MET Steamer
Validating Robonauts: Nanobot
Validating Robonauts: Neutral Beam
Validating Robonauts: D-D Fusion Inertial
Validating Robonauts: Nuclear Drill
Validating Robonauts: Helical Railgun
Validating Robonauts: Phase-Locked Diode Laser
Validating Robonauts: Lorentz-Propelled Microprobe
Validating Robonauts: Rock Splitter
Validating Robonauts: MagBeam
Validating Robonauts: Solar-Pumped MHD Exciplex Laser
Validating Robonauts: Quantum Cascade Laser
Validating Robonauts: Tungsten Resistojet
Validating Robonauts: MITEE Arcjet
ok


## Refineries

In [23]:
from jsonschema import validate, ValidationError
import json

refinery_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Refinery',
    'description': "One of the sides of a patent from the Refinery deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Refineries' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Air Eater' : { 'type': 'boolean' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'Support Requirements'],
    'additionalProperties': False,
}

save_schema(refinery_schema)

for card in refineries:
    try:
        print('Validating Refineries:', card['Name'])
        validate(card, refinery_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Refineries: Atomic Layer Deposition
Validating Refineries: Ilmenite Semiconductor Film
Validating Refineries: Basalt Fiber Spinning
Validating Refineries: Von Neumann Santa Claus Machine
Validating Refineries: CVD Molding
Validating Refineries: Carbonyl Volatilization
Validating Refineries: Carbo-Chlorination
Validating Refineries: Solar Carbotherm
Validating Refineries: Electroforming
Validating Refineries: Impact Mold Sinter
Validating Refineries: Fluidized Bed
Validating Refineries: Atmospheric Scoop
Validating Refineries: Foamglass Sintering
Validating Refineries: Laser-Heated Pedestal Growth
Validating Refineries: Froth Flotation
Validating Refineries: Femtochemistry
Validating Refineries: ISRU Sabatier
Validating Refineries: Biophytolytic Algal Farm
Validating Refineries: In-Situ Leaching
Validating Refineries: Termite Nest
Validating Refineries: Magma Electrolysis
Validating Refineries: Ionosphere Lasing
Validating Refineries: Supercritical Drying
Validating Refinerie

## Reactors

In [24]:
from jsonschema import validate, ValidationError
import json

movement_modifier_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Movement Modifiers',
    'description': 'A movement modifier triangle on a card.',
    'type': 'object',
    'properties': {
        'Thrust': {
            'description': 'The thrust modifier',
            'type': 'integer',
        },
        'Fuel Consumption': {
            'description': 'A string representation of the fuel consumption modifier as an integer or fraction.',
            'type': 'string',
        },
        'Solar': {
            'description': 'Whether the modifier applies solar.',
            'type': 'boolean',
        },
    },
    'required': ['Thrust', 'Fuel Consumption'],
}
save_schema(movement_modifier_schema)

reactor_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Reactor',
    'description': "One of the sides of a patent from the Reactor deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Reactors' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['X', '∿', '💣'] },
        },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Movement Modifiers': { '$ref': movement_modifier_schema['$id'] },
        'Air Eater': { 'type': 'boolean' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'Subtypes', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(reactor_schema)

for card in reactors:
    try:
        print('Validating Reactor:', card['Name'])
        validate(card, reactor_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Reactor: Cermet NERVA Fission
Validating Reactor: Pulsed NTR Fission
Validating Reactor: D-D Fusion Magneto-Inertial
Validating Reactor: H-B Fusion Reciprocating Plasmoid
Validating Reactor: D-T Fusion Tokamak
Validating Reactor: Antimatter GDM
Validating Reactor: D-T Gun Fusion
Validating Reactor: Macron Blowpipe Fusion
Validating Reactor: Lyman Alpha Trap
Validating Reactor: Free Radical Hydrogen Trap
Validating Reactor: Metallic Hydrogen
Validating Reactor: Fission-Augmented D-T Inertial Fusion
Validating Reactor: Mini-Mag RF Paul Trap
Validating Reactor: Ultracold Neutrons
Validating Reactor: Pebble Bed Fission
Validating Reactor: VCR Light Bulb Fission
Validating Reactor: Penning Trap
Validating Reactor: 3He-D Fusion Mirror Cell
Validating Reactor: Project Orion
Validating Reactor: Project Valkyrie
Validating Reactor: Rubbia Thin Film Fission Hohlraum
Validating Reactor: Positronium Bottle
Validating Reactor: Supercritical Water Fission
Validating Reactor: H-6Li Fusor
o

## Radiators

In [25]:
from jsonschema import validate, ValidationError
import json

radiator_orientation_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Radiator Orientation',
    'description': "One of the orientations of a Radiator card.",
    'type': 'object',
    'properties': {
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Therms': { 'type': 'integer' },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
    },
}
save_schema(radiator_orientation_schema)

radiator_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Radiator',
    'description': "One of the sides of a patent from the Radiator deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Radiators' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Light': { '$ref': radiator_orientation_schema['$id'] },
        'Heavy': { '$ref': radiator_orientation_schema['$id'] },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Light', 'Heavy'],
    'additionalProperties': False,
}
save_schema(radiator_schema)

for card in radiators:
    try:
        print('Validating Radiators:', card['Name'])
        validate(card, radiator_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Radiators: Bubble Membrane
Validating Radiators: Electrostatic Membrane
Validating Radiators: Dielectric X-Ray Window
Validating Radiators: Graphene Crystal X-Ray Mirror
Validating Radiators: ETHER Charged Dust
Validating Radiators: Curie Point
Validating Radiators: Li Heatsink Fountain
Validating Radiators: Thermochemical Heatsink Fountain
Validating Radiators: Magnetocaloric Refrigerator
Validating Radiators: Nuclear Fuel Spin Polarizer
Validating Radiators: Microtube Array
Validating Radiators: Marangoni Flow
Validating Radiators: Mo / Li Heat Pipe
Validating Radiators: Tin Droplet
Validating Radiators: Qu Tube
Validating Radiators: ANDR / In Dream Pipe
Validating Radiators: SS / NaK Pumped Loop
Validating Radiators: Hula-Hoop
Validating Radiators: Salt-Cooled Reflux Tube
Validating Radiators: Buckytube Filament
Validating Radiators: Steel / Pb-Bi Pumped Loop
Validating Radiators: Pulsating Heat Pipe
Validating Radiators: Ti / K Heat Pipe
Validating Radiators: Flux-Pinned

## Generators

In [26]:
from jsonschema import validate, ValidationError
import json

generator_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Generator',
    'description': "One of the sides of a patent from the Generator deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Generators' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Subtypes': {
            'type': 'array',
            'items': { 'enum': ['e', '⟛'] },
        },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Movement Modifiers': { '$ref': movement_modifier_schema['$id'] },
        'Air Eater': { 'type': 'boolean' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Mass', 'Rad-Hard', 'Subtypes', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(generator_schema)

for card in generators:
    try:
        print('Validating Generator:', card['Name'])
        validate(card, generator_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Generator: AMTEC Thermoelectric
Validating Generator: JTEC H2 Thermoelectric
Validating Generator: Brayton Turbine
Validating Generator: O'Meara LSP Paralens
Validating Generator: Cascade Photovoltaic
Validating Generator: Buckyball C60 Photovoltaic
Validating Generator: Cascade Thermoacoustic
Validating Generator: Dusty Plasma MHD
Validating Generator: Catalyzed Fission Scintillator
Validating Generator: Diamonoid Electro-Dynamic Tether
Validating Generator: Ericsson Engine
Validating Generator: Nanocomposite Thermoelectric
Validating Generator: Flywheel Compulsator
Validating Generator: Superconducting Adductor
Validating Generator: H2-O2 Fuel Cell
Validating Generator: Microbial Fuel Cell
Validating Generator: In-Core Thermionic
Validating Generator: Z-Pinch Microfission
Validating Generator: Magnetoshell Plasma Parachute
Validating Generator: Granular Rainbow Corral
Validating Generator: Marx Capacitor Bank
Validating Generator: Casimir Battery
Validating Generator: Opto

## GW Thrusters

In [27]:
from jsonschema import validate, ValidationError
import json

gw_thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'GW Thruster',
    'description': "The GW Thruster side of a card from the GW/TW Thruster deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'GW/TW Thrusters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'GW Thruster' },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(gw_thruster_schema)

tw_thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'TW Thruster',
    'description': "The TW Thruster side of a card from the GW/TW Thruster deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'GW/TW Thrusters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'TW Thruster' },
        'Spectral Type': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'items': {
                'oneOf': [
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Ability': { 'type': 'string' },
        'Future': {'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements', 'Future'],
    'additionalProperties': False,
}
save_schema(tw_thruster_schema)

gw_tw_thruster_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'GW/TW Thruster',
    'description': "One side of a card from the GW/TW Thruster deck.",
    'oneOf': [
        { '$ref': gw_thruster_schema['$id'] },
        { '$ref': tw_thruster_schema['$id'] },
    ]
}
save_schema(gw_tw_thruster_schema)

for i, card in enumerate(gwthrusters):
    try:
        if i % 2 == 0:
            print('Validating GW Thruster:', card['Name'])
            validate(card, gw_tw_thruster_schema)
        else:
            print('Validating TW Thruster:', card['Name'])
            validate(card, gw_tw_thruster_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating GW Thruster: Amat-Catalyzed Fission-Fusion
Validating TW Thruster: Amat-Initiated H-B Magnetic-Inertial
Validating GW Thruster: Dense Plasma H-B Focus Fusion
Validating TW Thruster: Crossfire H-B Focus Fusion
Validating GW Thruster: Levitated Dipole 6Li-H Fusion
Validating TW Thruster: Dusty Plasma
Validating GW Thruster: Mini-Mag Orion Z-Pinch Fission
Validating TW Thruster: Solem Medusa Tugged Orion
Validating GW Thruster: Salt-Water Zubrin
Validating TW Thruster: Zubrin-GDM
Validating GW Thruster: Spheromak 3He-D Magnetic Fusion
Validating TW Thruster: Colliding FRC 3He-D Fusion
Validating GW Thruster: VISTA D-T Inertial Fusion
Validating TW Thruster: Daedalus 3He-D Inertial Fusion
ok


## Freighters

In [28]:
from jsonschema import validate, ValidationError
import json

basic_freighter_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Basic Freighter',
    'description': "The basic Freighter side of a card from the Freighters patent deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Freighters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'Freighter' },
        'Spectral Type': {
            'enum': ['Any', 'C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Supports Provided': {
            'type': 'array',
            'item': { 'enum': ['e', '⟛',  'X', '∿', '💣'] },
        },
        'Load-Limit': { 'type': 'integer' },
        'Factory Loading Only': { 'type': 'boolean' },
        'Bonus Pivots': { 'type': 'integer' },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Type', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Load-Limit', 'Supports Provided'],
    'additionalProperties': False,
}
save_schema(basic_freighter_schema)

freighter_fleet_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Freighter Fleet',
    'description': "The Freighter Fleet side of a card from the Freighters patent deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Freighters' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Type': { 'const': 'Freighter Fleet' },
        'Spectral Type': {
            'enum': ['Any', 'C', 'S', 'M', 'V', 'D', 'H'],
        },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Supports Provided': {
            'type': 'array',
            'item': { 'enum': ['e', '⟛',  'X', '∿', '💣'] },
        },
        'Load-Limit': { 'type': 'integer' },
        'Factory Loading Only': { 'type': 'boolean' },
        'Bonus Pivots': { 'type': 'integer' },
        'Ability': { 'type': 'string' },
        'Future': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Type', 'Spectral Type', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Load-Limit', 'Supports Provided', 'Future'],
    'additionalProperties': False,
}
save_schema(freighter_fleet_schema)

freighter_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Freighter',
    'description': "One side of a card from the Freighter deck.",
    'oneOf': [
        { '$ref': basic_freighter_schema['$id'] },
        { '$ref': freighter_fleet_schema['$id'] },
    ]
}
save_schema(freighter_schema)

for i, card in enumerate(freighters):
    try:
        print('Validating Freighter:', card['Name'])
        validate(card, freighter_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Freighter: Fission-Heated Steam
Validating Freighter: Fission GCR
Validating Freighter: Fusion Fragment Sail
Validating Freighter: Antiproton Sail and Harvester
Validating Freighter: HIIPER Beam Rider
Validating Freighter: Magnetic Mirror Beam Rider
Validating Freighter: Inflatable Solar-Heated
Validating Freighter: Archimedes Palmer Lens
Validating Freighter: Poodle Steam
Validating Freighter: D-Nanotube Dirt Launcher
Validating Freighter: Rotary Dirt Launcher
Validating Freighter: KESTS Hoop Dirt Launcher
Validating Freighter: Z-Pinch D-T / 6Li Fusion
Validating Freighter: Z-Pinch 3He-D Target Fusion
ok


## Bernals

In [29]:
from jsonschema import validate, ValidationError
import json

bernal_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    'title': 'Bernal',
    'description': "The basic Bernal side of a card from the Bernals patent deck.",
    'type': 'object',
    'properties': {
        'Name': { 'type': 'string' },
        'Deck': { 'const': 'Bernals' },
        'Side': { 'enum': [ 'Front', 'Reverse' ] },
        'Promotion Colony': {
            'enum': ['C', 'S', 'M', 'V', 'D', 'H', 'Push', 'Atmospheric', 'Submarine', 'Astrobiology'],
        },
        'Mass': { 'type': 'integer' },
        'Rad-Hard': { 'type': 'integer' },
        'Thrust Triangle': { '$ref': thrust_triangle_schema['$id'] },
        'Support Requirements': {
            'type': 'array',
            'item': {
                'oneOf': [
                    { '$ref': reactor_support_requirement_schema['$id'] },
                    { '$ref': generator_support_requirement_schema['$id'] },
                    { '$ref': cooling_support_requirement_schema['$id'] },
                ]
            },
        },
        'Ability': { 'type': 'string' },
    },
    'required': ['Name', 'Deck', 'Side', 'Promotion Colony', 'Mass', 'Rad-Hard', 'Thrust Triangle', 'Support Requirements'],
    'additionalProperties': False,
}
save_schema(bernal_schema)

for i, card in enumerate(bernals):
    try:
        print('Validating Bernal:', card['Name'])
        validate(card, bernal_schema)
    except ValidationError as e:
        print('Error validating %s: %s' % (card['Deck'],  card['Name']))
        print(json.dumps(card, ensure_ascii=False, indent=4))
        raise e
print('ok')

Validating Bernal: GEO Elevator Bernal
Validating Bernal: Space Elevator Lab
Validating Bernal: L1 Climate Control Bernal
Validating Bernal: Climate Control Lab
Validating Bernal: L2 Collimator Bernal
Validating Bernal: Collimator Lab
Validating Bernal: L3 Lofstrom Loop Microgravity
Validating Bernal: Lofstrom Loop Microgravity Lab
Validating Bernal: L4 Antimatter Factory
Validating Bernal: Antimatter Lab
Validating Bernal: L4s Pharmaceutics Bernal
Validating Bernal: Pharmaceutics Lab
Validating Bernal: L5 Solar Cell Factory
Validating Bernal: Solar Cell Lab
Validating Bernal: L5s Cancer Hospital
Validating Bernal: Cancer Lab
Validating Bernal: SSO Diplomatic
Validating Bernal: Diplomatic Lab
Validating Bernal: Tourism Cycler
Validating Bernal: Tourism Hotel
ok
